### Import

In [1]:
import gzip
import json
import random
import time
from nltk import word_tokenize
from tqdm import tqdm
import contractions

# Imports for the synonym dictionary 
import requests
from bs4 import BeautifulSoup
from nltk.wsd import lesk

#wordnet imports
#importing NLTK corpus synonym dictionary, pos-tagging and checking the outputs
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
import random
from sklearn.pipeline import Pipeline

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

#Word2Vec imports
import gensim.models

In [2]:
#wordnet downloads
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
random.seed(42)

### Read in the data

In [4]:
#here
def reading_in(path):
    thing=[]
    for line in gzip.open(path):
        review_data = json.loads(line)
        subthing = dict()
        for key in review_data:
            subthing[key]= review_data[key]
        thing.append(subthing)
    return thing

In [5]:
train = reading_in('../data/classification/music_reviews_train.json.gz')
dev = reading_in('../data/classification/music_reviews_dev.json.gz') # do not use!
#test = reading_in('../data/classification/music_reviews_test_masked.json.gz') #do not use!
train = shuffle(train, random_state=42)

In [6]:
train_x = [i['reviewText'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]
train_y =  [i['sentiment'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]

dev_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in dev]
dev_y = [i['sentiment'] for i in dev]

In [ ]:
pos_neg = [sum([i =="positive" for i in train_y]),sum([i != "positive" for i in train_y])]

In [7]:
import matplotlib.pyplot as plt
plt.bar(x=["pos","neg"], height = pos_neg) 

NameError: name 'pos_neg' is not defined

In [8]:
###split into pos and neg
train_x_pos = []
train_x_neg = []
train_y_pos = []
train_y_neg = []
for i in range(len(train_x)):
    if train_y[i] == "positive":
        train_x_pos.append(train_x[i])
        train_y_pos.append(train_y[i])
    else:
        train_x_neg.append(train_x[i])
        train_y_neg.append(train_y[i])
train_x_neg.extend(train_x_pos[0:len(train_x_neg)])
train_y_neg.extend(train_y_pos[0:len(train_y_neg)])
balanced_x_train = train_x_neg
balanced_y_train = train_y_neg

In [9]:
len(balanced_x_train)

78424

In [ ]:
len(train_x_neg)

NameError: name 'train_x_neg' is not defined

### Preparing the data

In [ ]:
#old tokenizer
#train_x_back = [' '.join(sen) for sen in train_x_split] may not need convertes tokenised back to single string

In [10]:
#Tokenizer
def tokenize_corpus(corpus):
    return [word_tokenize(contractions.fix(s)) for s in corpus]

In [ ]:
train_x_split = tokenize_corpus(balanced_x_train)

# Webscrape synonyms

In [ ]:
##Alternative to PyDictionary
##THIS IS COPIED FROM SLACK
# https://stackoverflow.com/questions/52910297/pydictionary-word-has-no-synonyms-in-the-api

def synonyms(term):
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.text, 'lxml')
    soup.find('section', {'class': 'css-17ofzyv e1ccqdb60'})
    return [span.text for span in soup.findAll('a', {'class': 'css-1kg1yv8 eh475bn0'})] # 'css-1gyuw4i eh475bn0' for less relevant synonyms

word = "us"
print(synonyms(word))

['them ', 'you ']


In [ ]:
dont_replace = ['am','are','be','been','being','did','do','does','had','has','have','having','is','was','were','will','I ', 'it ', 'she ', 'sie ', 'they ', 've ', 'xe ', 'you ', 'ze ', 'zie ','he',"you","we","mine","hers","his","yours","its","ours","theirs","your","our","oneself","oneselves","yourself","yourselves","himself","herself","himself","myself","themselves","itself","ourselves","ourself","us","them"] #Used to avoid changing function and pronouns words
random.seed(43)
def gen_synth_web(sentence,prob = 0.5):
    new_sen = []
    for word in sentence:
        if word in dont_replace:
            new_sen.append(word)
        else:
            threshold = random.random()
            if prob > threshold:
                #print(threshold) can remove later kept for debugging
                syn = synonyms(word)
                if len(syn) != 0:
                    #print("selected", word) can remove later kept for debugging
                    new_sen.append(random.choice(syn))
                else:new_sen.append(word)
            else:
                new_sen.append(word)
    return new_sen

In [ ]:
print(train_x_split[12])

NameError: name 'train_x_split' is not defined

In [ ]:
print(gen_synth_web(train_x_split[12]))

NameError: name 'gen_synth_web' is not defined

In [ ]:
#for i in tqdm(train_x_split[:1000]):#running with 0:1000 dont uncomment as will mess up file 
   # with open ("../data/synthetic1.txt","a") as f:
       # f.write(' '.join(gen_synth_web(i))+"\n")
       # time.sleep(3)

  1%|▏         | 13/1000 [01:18<1:39:39,  6.06s/it]


KeyboardInterrupt: 

#### Example of webscrape synthetic sentence 

In [ ]:
print(train_x_split[6])

['I', 'love', 'all', 'of', 'his', 'music', '!', '!']


In [ ]:
print(gen_synth_web(train_x_split[6]))

['I', 'passion ', 'fully ', 'of', 'his', 'music', '!', '!']


# Wordnet

In [ ]:
relevant = ["NN","NNS","RB","RBR","RBS","VB","VBG","VBD","VBN","VBP","VBZ","JJ","JJR","JJS"]#The wordclasses that can be exchanged with synonyms (not functional words) #maybe not RB
wordnet_tag_map = {'NN': ['n'],'NNS': ['n'],'JJ': ['a'],'JJR': ['a'],'JJS': ['a'],'RB': ['r'],'RBR': ['r'],'RBS': ['r'], 'VB': ['v'], 'VBG': ['v'], 'VBD': ['v'], 'VBN': ['v'], 'VBP': ['v'], 'VBZ': ['v']} #Making pos-tags coincide
aux_verbs = ['am','are','be','been','being','did','do','does','had','has','have','having','is','was','were','will'] #Used to avoid changing function words
def change_wordnet(sentence):
    sentence = nltk.pos_tag(sentence)    #pos-tagging sentence here
    new_sentence = [] #to append to and then return as final sentence
    for word in sentence:
        if word[1] in relevant and word[0] not in aux_verbs: #Not changing functional words
            tag = wordnet_tag_map[word[1]]
            chosen = lesk(sentence, word[0], tag[0]) #from all synsets choose the most likely in a smart way, thanks to library!
            if chosen is None:
                new_sentence.append(word[0]) #sometimes no synset is chosen so we just give the new sentence the original word
            else:
                options = chosen.lemma_names()
                if word[0] in options:
                    options.remove(word[0])
                if len(options)>0:
                    chosen_w = random.choice(options)
                    if "_" in chosen_w:
                        for w in chosen_w.split("_"):
                            new_sentence.append(w)
                    else:
                        new_sentence.append(word[0])
                else:
                    new_sentence.append(word[0])
        else:
            new_sentence.append(word[0])
    return new_sentence

#### Example of wordnet synthetic sentence 

In [ ]:
print(train_x_split[10])

['The', 'book', 'Think', '&', 'Grow', 'Rich', 'is', 'a', 'must', 'own', '.', 'It', 'is', 'book', 'to', 'be', 'read', 'multiple', 'times', '.', 'That', 'being', 'said', 'the', 'particular', 'reader', 'of', 'this', 'audio', 'book', 'is', 'very', 'monotone', '.', 'He', 'does', 'not', 'pause', 'at', 'the', 'end', 'of', 'sentence', 'or', 'even', 'a', 'paragraph', '.', 'You', 'can', 'hear', 'the', '``', 'paah', "''", 'noises', 'into', 'the', 'microphone', 'every', 'few', 'seconds', '.', 'The', 'narrator', 'has', 'a', 'strong', 'voice', 'it', 'is', 'just', 'emotionless', '.', 'It', 'reminded', 'me', 'of', 'when', 'a', 'teacher', 'would', 'call', 'on', 'a', 'student', 'to', 'read', 'aloud', '...', 'but', 'the', 'student', 'is', 'disinterested', 'a', 'tries', 'to', 'blow', 'through', 'it', 'as', 'quickly', 'and', 'unenthusiastically', 'as', 'possible', 'so', 'he', 'can', 'go', 'back', 'to', 'day', 'dreaming', '.', 'Bottomline', ':', 'Buy', 'the', 'book', ',', 'buy', 'the', 'audio', 'book', 'jus

In [ ]:
print(change_wordnet(train_x_split[10]))

['The', 'book', 'Think', '&', 'Grow', 'Rich', 'is', 'a', 'must', 'own', '.', 'It', 'is', 'book', 'to', 'be', 'read', 'multiple', 'times', '.', 'That', 'being', 'said', 'the', 'particular', 'reader', 'of', 'this', 'audio', 'book', 'is', 'very', 'monotone', '.', 'He', 'does', 'not', 'pause', 'at', 'the', 'end', 'of', 'sentence', 'or', 'even', 'a', 'paragraph', '.', 'You', 'can', 'take', 'heed', 'the', '``', 'paah', "''", 'noises', 'into', 'the', 'microphone', 'every', 'few', 'second', 'gear', '.', 'The', 'narrator', 'has', 'a', 'strong', 'voice', 'it', 'is', 'just', 'emotionless', '.', 'It', 'reminded', 'me', 'of', 'when', 'a', 'teacher', 'would', 'call', 'in', 'on', 'a', 'student', 'to', 'read', 'aloud', '...', 'but', 'the', 'student', 'is', 'disinterested', 'a', 'tries', 'to', 'blow', 'through', 'it', 'as', 'quickly', 'and', 'unenthusiastically', 'as', 'possible', 'so', 'he', 'can', 'go', 'back', 'to', 'sidereal', 'day', 'dreaming', '.', 'Bottomline', ':', 'Buy', 'the', 'book', ',', 'b

#### write to file

In [ ]:
wn_x_train = [change_wordnet(i) for i in train_x_split]

In [ ]:
len(wn_x_train)

78424

In [ ]:
with open("../data/synthetic/wordnet.csv",'w') as f:
    f.writelines('\t'.join(s) + '\n' for s in wn_x_train)

# Word2vec

In [ ]:
gooEmbs.most_similar('walk')

[('walking', 0.7361534237861633),
 ('walks', 0.7007260918617249),
 ('walked', 0.6831838488578796),
 ('stroll', 0.6208134889602661),
 ('Walking', 0.5674901604652405),
 ('Walk', 0.558607280254364),
 ('wander', 0.5585978627204895),
 ('jog', 0.5467169284820557),
 ('saunter', 0.5451485514640808),
 ('amble', 0.5250112414360046)]

In [ ]:
gooEmbs = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-50k.bin', binary=True)
print('loading finished')
relevant = ["NN","NNS","RB","RBR","RBS","VB","VBG","VBD","VBN","VBP","VBZ","JJ","JJR","JJS"]#The wordclasses that can be exchanged with synonyms (not functional words) #maybe not RB
aux_verbs = ['am','are','be','been','being','did','do','does','had','has','have','having','is','was','were','will'] #Used to avoid changing function words
def change_word2vec(original_sentence, prob = 0):
    sentence = nltk.pos_tag(original_sentence)    #pos-tagging sentence here
    new_sentence = [] #to append to and then return as final sentence
    for word in sentence:
        if word[1] in relevant and word[0] not in aux_verbs:
            if random.random() > prob:
                try:
                    syno = gooEmbs.most_similar(word[0]) #taking the most similar word from word2vec
                    if syno[0] != word[0]: # think this is supperficial since you cannot get the same word in word2vec
                        new_sentence.append(syno[0])
                    else:
                        new_sentence.append(syno[1])
                except:
                    new_sentence.append(word[0])
            else: 
                new_sentence.append(word[0])
        else:
            new_sentence.append(word[0])
    return new_sentence

loading finished


In [11]:
# With a threshold and without the check of same word

gooEmbs = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-50k.bin', binary=True)
print('loading finished')
relevant = ["NN","NNS","RB","RBR","RBS","VB","VBG","VBD","VBN","VBP","VBZ","JJ","JJR","JJS"]#The wordclasses that can be exchanged with synonyms (not functional words) #maybe not RB
aux_verbs = ['am','are','be','been','being','did','do','does','had','has','have','having','is','was','were','will'] #Used to avoid changing function words
def change_word2vec(original_sentence, prob = 0, threshold = .5):
    sentence = nltk.pos_tag(original_sentence)    #pos-tagging sentence here
    new_sentence = [] #to append to and then return as final sentence
    for word in sentence:
        if word[1] in relevant and word[0] not in aux_verbs:
            if random.random() > prob:
                try:
                    syno = gooEmbs.most_similar(word[0]) #taking the most similar word from word2vec
                    syno_pos = []
                    # method 1: 
                    for s in syno:
                        if s[1] > threshold:
                            syno_pos.append(s)
                        else: 
                            break # because all the words are sorted we know that if one word is under 60% all the others after are also
                    new_word = random.choice(syno_pos)[0]
                    new_sentence.append(new_word)
                  #  if "_" in new_word:
                   #     for w in chosen_w.split("_"):
                    #        new_sentence.append(w)
                    #else: new_sentence.append(new_word)
                except:
                    new_sentence.append(word[0])
            else: 
                new_sentence.append(word[0])
        else:
            new_sentence.append(word[0])
    return new_sentence

loading finished


In [ ]:
ori = train_x_split[3]
t7= change_word2vec(train_x_split[3], threshold=0.7)
t6 =change_word2vec(train_x_split[3], threshold=0.6)
t5 = change_word2vec(train_x_split[3], threshold=0.5)
t4 = change_word2vec(train_x_split[3], threshold=0.4)
t3 = change_word2vec(train_x_split[3], threshold=0.3)

for i in range(len(ori)):
    print(f'o: {ori[i]}, .7: {t7[i]}, .6: {t6[i]}, .5: {t5[i]}, .4 {t4[i]}, .3: {t3[i]}')

o: Nickleback, .7: Nickleback, .6: Nickleback, .5: Nickleback, .4 Nickleback, .3: Nickleback
o: is, .7: is, .6: is, .5: is, .4 is, .3: is
o: Clear, .7: Clear, .6: Clear, .5: Clear, .4 Visible, .3: Clean
o: Channel, .7: Channel, .6: Channel, .5: Channel, .4 Channel, .3: Channel
o: 's, .7: 's, .6: 's, .5: 's, .4 's, .3: 's
o: best, .7: best, .6: finest, .5: terrific, .4 finest, .3: worst
o: friend, .7: pal, .6: uncle, .5: colleague, .4 colleague, .3: dear_friend
o: :, .7: :, .6: :, .5: :, .4 :, .3: :
o: their, .7: their, .6: their, .5: their, .4 their, .3: their
o: music, .7: classical_music, .6: musicians, .5: classical_music, .4 musicians, .3: musical
o: is, .7: is, .6: is, .5: is, .4 is, .3: is
o: ordinary, .7: ordinary, .6: Ordinary, .5: everyday, .4 extraordinary, .3: humdrum
o: ,, .7: ,, .6: ,, .5: ,, .4 ,, .3: ,
o: dull, .7: boring, .6: monotonous, .5: bland, .4 monotonous, .3: duller
o: ,, .7: ,, .6: ,, .5: ,, .4 ,, .3: ,
o: and, .7: and, .6: and, .5: and, .4 and, .3: and
o: abso

In [ ]:
#new_x = []
#for  i in train_x[:1000]:
    #new_x.append(change_word2vec(i))

In [ ]:
print(train_x_split[6])
print(gen_synth_web(train_x_split[6]))
print(change_wordnet(train_x_split[6]))
print(change_word2vec(train_x_split[6]))

['I', 'love', 'all', 'of', 'his', 'music', '!', '!']
['ve ', 'love', 'group ', 'of', 'his', 'music', '!', '!']
['I', 'have_intercourse', 'all', 'of', 'his', 'music', '!', '!']


UnboundLocalError: local variable 'sentence' referenced before assignment

In [ ]:
 gooEmbs.most_similar('woman')

[('man', 0.7664012908935547),
 ('girl', 0.7494640946388245),
 ('teenage_girl', 0.7336829304695129),
 ('teenager', 0.6317085027694702),
 ('lady', 0.6288785934448242),
 ('teenaged_girl', 0.6141784191131592),
 ('mother', 0.6076306104660034),
 ('policewoman', 0.6069462299346924),
 ('boy', 0.5975907444953918),
 ('Woman', 0.5770983099937439)]

# Baseline

In [ ]:
pipeline = Pipeline([
    ('CountVectorizer',CountVectorizer(ngram_range=(1, 2))),
    ('Tfidf',TfidfTransformer()),
    ('clf', MultinomialNB(alpha=1, fit_prior=False))
])

In [ ]:
size = [500, 1000, 2500, 5000, 10000, 20000, 30000, 50000, 60000, 70000, 80000, 90000, 100000]
train_acc = []
dev_acc = []
for i in size:
    pipeline.fit(balanced_x_train[:i],balanced_y_train[:i] )
    pred_train = pipeline.predict(balanced_x_train)
    train_acc.append(f1_score(balanced_x_train, pred_train, average='macro'))
    pred = pipeline.predict(dev_x)
    dev_acc.append(f1_score(dev_y, pred, average='macro'))
    print(f'done with size {i}')


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
plt.plot(size, dev_acc)

In [ ]:
plt.plot(size, train_acc)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b88dfe01-c7e1-473c-bcfd-798313fc6522' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>